In [1]:
import pandas as pd
import os, csv
import numpy as np
from sqlalchemy import create_engine
from itertools import chain

In [2]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(', ')))

In [ ]:
# flat = pd.DataFrame({
#     'Manufacturer':np.repeat(df1['Manufacturer'], lens),
#     'Series':np.repeat(df1['Series'], lens),
#     'Color':np.repeat(df1['Color'], lens),
#     'Flat RCCP':chainer(df1['Flat RCCP'])
# })

In [3]:
netflix_data = os.path.join('Resources','netflix','netflix_titles.csv')
netflix = pd.read_csv(netflix_data)
netflix_df = pd.DataFrame(netflix)
netflix_df = netflix_df.dropna()

In [14]:
cleaned_stage1_netflix = netflix_df[['show_id','type','title','director','cast','release_year','description']].copy()

lens = cleaned_stage1_netflix['cast'].str.split(', ').map(len)

cleaned_stage2_netflix = pd.DataFrame({
    'show_id':np.repeat(cleaned_stage1_netflix['show_id'],lens),
    'type':np.repeat(cleaned_stage1_netflix['type'],lens),
    'title':np.repeat(cleaned_stage1_netflix['title'],lens),
    'director':np.repeat(cleaned_stage1_netflix['director'],lens),
    'full_name':chainer(cleaned_stage1_netflix['cast']),
    'release_year':np.repeat(cleaned_stage1_netflix['release_year'],lens),
    'description':np.repeat(cleaned_stage1_netflix['description'],lens)
})

In [25]:
cleaned_stage3_netflix = cleaned_stage2_netflix #[cleaned_stage2_netflix['release_year'] == 2019]
cleaned_stage3_netflix['title'] = cleaned_stage3_netflix['title'].str.strip().str.title()
cleaned_stage3_netflix['full_name'] = cleaned_stage3_netflix['full_name'].str.strip().str.title()
ids = []

for x in range(len(cleaned_stage3_netflix['show_id'])):
    ids.append(x)

cleaned_stage3_netflix.insert(0,'id',ids)

final_netflix_df = cleaned_stage3_netflix

In [26]:
final_netflix_df


,id,show_id,type,title,director,full_name,release_year,description
0,0,81145628,Movie,Norm Of The North: King Sized Adventure,"Richard Finn, Tim Maltby",Alan Marriott,2019,Before planning an awesome wedding for his gra...
0,1,81145628,Movie,Norm Of The North: King Sized Adventure,"Richard Finn, Tim Maltby",Andrew Toth,2019,Before planning an awesome wedding for his gra...
0,2,81145628,Movie,Norm Of The North: King Sized Adventure,"Richard Finn, Tim Maltby",Brian Dobson,2019,Before planning an awesome wedding for his gra...
0,3,81145628,Movie,Norm Of The North: King Sized Adventure,"Richard Finn, Tim Maltby",Cole Howard,2019,Before planning an awesome wedding for his gra...
0,4,81145628,Movie,Norm Of The North: King Sized Adventure,"Richard Finn, Tim Maltby",Jennifer Cameron,2019,Before planning an awesome wedding for his gra...
...,...,...,...,...,...,...,...,...
6213,29384,80126599,TV Show,Leyla And Mecnun,Onur Ünlü,Cengiz Bozkurt,2014,Destiny brings Mecnun and Leyla together as ne...
6213,29385,80126599,TV Show,Leyla And Mecnun,Onur Ünlü,Osman Sonant,2014,Destiny brings Mecnun and Leyla together as ne...
6213,29386,80126599,TV Show,Leyla And Mecnun,Onur Ünlü,Köksal Engür,2014,Destiny brings Mecnun and Leyla together as ne...
6213,29387,80126599,TV Show,Leyla And Mecnun,Onur Ünlü,Ege Tanman,2014,Destiny brings Mecnun and Leyla together as ne...


In [11]:
sag_data = os.path.join('Resources','SAG','screen_actor_guild_awards.csv')
sag = pd.read_csv(sag_data)
sag_df = pd.DataFrame(sag)

sag_df
years = []

for x in range(len(sag_df['year'])-1):
    years.append(sag_df['year'].str.split(' ')[x][0])

In [27]:
sag_df['years_cleaned'] = pd.Series(years)
cleaned_stage1_sag = sag_df[['years_cleaned','category','full_name','show','won']].copy()
# cleaned_stage1_sag = cleaned_stage1_sag[cleaned_stage1_sag['years_cleaned']=='2019']
cleaned_stage2_sag = cleaned_stage1_sag.dropna().reset_index()
cleaned_stage2_sag['full_name'] = cleaned_stage2_sag['full_name'].str.title().str.strip()
cleaned_stage2_sag['show'] = cleaned_stage2_sag['show'].str.title().str.strip()
cleaned_stage3_sag = cleaned_stage2_sag.rename(columns={'years_cleaned':'year','index':'id','show':'title'})
final_sag_df = cleaned_stage3_sag.set_index('id')
final_sag_df.head(30)

,year,category,full_name,title,won
id,,,,,
5,2020,FEMALE ACTOR IN A LEADING ROLE,Cynthia Erivo,Harriet,False
6,2020,FEMALE ACTOR IN A LEADING ROLE,Scarlett Johansson,Marriage Story,False
7,2020,FEMALE ACTOR IN A LEADING ROLE,Lupita Nyong'O,Us,False
8,2020,FEMALE ACTOR IN A LEADING ROLE,Charlize Theron,Bombshell,False
9,2020,FEMALE ACTOR IN A LEADING ROLE,Renée Zellweger,Judy,True
10,2020,MALE ACTOR IN A LEADING ROLE,Christian Bale,Ford V Ferrari,False
11,2020,MALE ACTOR IN A LEADING ROLE,Leonardo Dicaprio,Once Upon A Time...In Hollywood,False
12,2020,MALE ACTOR IN A LEADING ROLE,Adam Driver,Marriage Story,False
13,2020,MALE ACTOR IN A LEADING ROLE,Taron Egerton,Rocketman,False


In [31]:
merge_test = pd.merge(final_netflix_df, final_sag_df, on='title')
merge_test = merge_test[merge_test['full_name_x'] == merge_test['full_name_y']]
merge_test_stage2 = merge_test[['id','show_id','type','director','full_name_x','release_year','description','category','won']]
merge_test_stage3 = merge_test_stage2.rename(columns={'full_name_x':'full_name'})
merge_test_stage3

,id,show_id,type,director,full_name,release_year,description,category,won
2,279,80201906,Movie,Ryan Coogler,Chadwick Boseman,2018,"T'Challa, the superpowered new leader of the h...",CAST IN A MOTION PICTURE,True
21,280,80201906,Movie,Ryan Coogler,Michael B. Jordan,2018,"T'Challa, the superpowered new leader of the h...",CAST IN A MOTION PICTURE,True
39,282,80201906,Movie,Ryan Coogler,Danai Gurira,2018,"T'Challa, the superpowered new leader of the h...",CAST IN A MOTION PICTURE,True
56,283,80201906,Movie,Ryan Coogler,Martin Freeman,2018,"T'Challa, the superpowered new leader of the h...",CAST IN A MOTION PICTURE,True
64,284,80201906,Movie,Ryan Coogler,Daniel Kaluuya,2018,"T'Challa, the superpowered new leader of the h...",CAST IN A MOTION PICTURE,True
...,...,...,...,...,...,...,...,...,...
3953,28777,607931,Movie,Jocelyn Moorhouse,Kate Nelligan,1995,A conflicted young woman spends the summer wit...,CAST IN A MOTION PICTURE,False
3958,28778,607931,Movie,Jocelyn Moorhouse,Alfre Woodard,1995,A conflicted young woman spends the summer wit...,CAST IN A MOTION PICTURE,False
4018,28784,607931,Movie,Jocelyn Moorhouse,Maya Angelou,1995,A conflicted young woman spends the summer wit...,CAST IN A MOTION PICTURE,False
4051,28788,607931,Movie,Jocelyn Moorhouse,Jean Simmons,1995,A conflicted young woman spends the summer wit...,CAST IN A MOTION PICTURE,False
